# Pulsar Actuator Control using CANoverUSB (Two Targets)
This notebook demonstrates how to control Pulsar actuators using the CANoverUSB interface. We will go through the steps of connecting to the USB-CAN adapter, initializing the actuators, configuring them, and finally running them.

In [ ]:
# Import necessary modules
from pcp_api.PulsarActuator import PulsarActuator
from pcp_api.can_over_usb import CANoverUSB
from pprint import pprint
from time import sleep

## Define Constants and Feedback Function
We define the CSP addresses for the actuators and a feedback function to handle actuator feedback.

In [ ]:
# Example CSP addresses for two actuators
CSP_ADDRESSES = [0x10, 0x11]

## Define a feedback callback function

This function will be called automatically when feedback is received from the actuator.

In [ ]:
def actuator_feedback(address: int, feedback: dict):
    position = feedback.get(PulsarActuator.PCP_Items.POSITION_FB, None)
    print(f"Actuator 0x{address:X} position: {position:.2f} rad/s")

## Connect to USB-CAN Adapter
We connect to the USB-CAN adapter. The port is auto-detected.

In [ ]:
# Auto-detect the port
port = CANoverUSB.get_port()
print(f"Connecting to {port}")
adapter = CANoverUSB(port)

## Initialize and Configure Actuators
We initialize the actuators, connect to them, and configure their feedback settings.

In [ ]:
actuators = []

for address in CSP_ADDRESSES:
    # Create a PulsarActuator instance.
    actuator = PulsarActuator(adapter, address)

    # Attempt to connect to the actuator.
    if not actuator.connect():
        print(f"Could not connect to the actuator {actuator.address}")
        adapter.close()
        raise SystemExit(1)

    print(f"Connected to the actuator {actuator.address}")

    # Configure high-frequency feedback to report position at 10 Hz.
    actuator.setHighFreqFeedbackItems([
        PulsarActuator.PCP_Items.POSITION_FB,
    ])
    actuator.setHighFreqFeedbackRate(actuator.Rates.RATE_10HZ)

    # Disable low-frequency feedback.
    actuator.setLowFreqFeedbackRate(actuator.Rates.DISABLED)

    # Set the actuator to SPEED mode.
    actuator.change_mode(PulsarActuator.Mode.SPEED)

    # Register the feedback callback function.
    actuator.set_feedback_callback(actuator_feedback)

    actuators.append(actuator)
    

## Set Setpoints and Start Actuators
We set different speeds for each actuator and start them.

In [ ]:
# Set different speeds for each actuator
actuators[0].change_setpoint(0.2)
actuators[1].change_setpoint(0.3)

# Start all actuators
for actuator in actuators:
    actuator.start()

## Run and Cleanup
We run the actuators for a short period and then clean up by disconnecting the actuators and closing the adapter.

In [ ]:
try:
    sleep(0.1)  # Let feedback trigger
except KeyboardInterrupt:
    pass
finally:
    for actuator in actuators:
        actuator.disconnect()
    sleep(0.1)
    adapter.close()